In [39]:
!pip3 install openai
!pip3 install asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import pandas

questions = pandas.read_csv('https://raw.githubusercontent.com/bamartin1618/1WeekDataset/main/Question_Classification_Dataset.csv')
question_sample = questions[questions.columns[1]].sample(50).to_numpy()

# Approach 1: Single Requests
<p>The following code answers fifty questions using the GPT-3 API, sending one request at a time.</p>
<p>Straightforward but slow.</p>

In [41]:
import os
import openai
import time
import aiohttp
import asyncio

openai.api_key = str(os.getenv('GPT_KEY'))

t0 = time.time()

answers = []

for question in question_sample:

  response = openai.Completion.create(
      model='text-ada-001',
      prompt=question,
      temperature=0.7,
      max_tokens=100,
      top_p=1
  )

  answers.append(response['choices'][0]['text'])

t1 = time.time()
print(t1-t0)
print(answers)

13.090801239013672
['\n\nLinux is a term used to describe the design of a computer system from the weeks of development starting in 1998 through to the present. It refers to the "Linux kernel" and the "linux-sys Thank you for choosing Linux!', '\n\nThe sense of smell becomes active first.', '\n\nThe tropical rain forest distributions are mainly located in the Americas, Africa, and Australia.', "\n\nThe percentage of children between the ages of two and eleven who watch `The Simpsons' is about 77%.", '\n\nOzzy Osbourne was born on December 5, 1965.', "\n\nThere is no one answer to this question as there are many reasons why one might want to stop or slow down aging. Some reasons might be to prevent serious health problems, to enjoy younger-looking skin and hair, or to feel older-looking skin and hair without any doctor's advice. Other reasons might be to prevent some types of death, such as old age or natural death. If you want to stop or slow down aging, you might need to experiment an

# Approach 2: Concurrent Requests
<p>It's really easy to make concurrent requests with the GPT-3 API, and there's a huge boost in speed.</p>
<p>You can do it by taking the endpoint that the GPT Playground sends requests to through the Network Tab in Developer Tools. The rest is simple.</p>

In [42]:
!pip3 install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
from pickle import TUPLE2
import nest_asyncio
nest_asyncio.apply()

async def get_text(session, url, params):

    async with session.post(url, json = params) as resp:
        text = await resp.json()
        return text['choices'][0]['text']

async def answer_questions():

  async with aiohttp.ClientSession(headers = {'authorization' : f"Bearer {os.getenv('GPT_TOKEN')}"}) as session:

    jobs = []
    
    for question in questions:
      url = 'https://api.openai.com/v1/engines/text-ada-001/completions'
      jobs.append(asyncio.ensure_future(get_text(session, url, {'prompt' : 'question', 'temperature' : 0.7, 'max_tokens' : 100, 'top_p' : 1})))

    responses = await asyncio.gather(*jobs)

  return responses

t2 = time.time()
answers = asyncio.run(answer_questions())
t3 = time.time()

print(t1-t0)
print(answers)

13.090801239013672
['\n\nWhat is the difference between a forward-facing engine and a back-facing engine?\n\nA forward-facing engine is a engine that is facing the front of the car, while a back-facing engine is facing the back of the car.', ' I asked\n\nWhat are your favorite and least favorite plant types?\n\nI love all plant types!', '\n\nWhat do you do when you are sick?\n\nI go to the hospital.', ')\n\nWhat is the name of the protagonist in the novel?\n\nThe protagonist in the novel is John.', '\n\nWhat is the difference between a plan and a budget?\n\nA budget is a plan for how money will be used in the budget, while plan is a plan of how money will be used, usually one prepared with help of a financial advisor.']


In [44]:
print(f'Single Requests (Time to Complete): {t1-t0} seconds')
print(f'Concurrent Requests (Time to Complete): {t3-t2} seconds')

Single Requests (Time to Complete): 13.090801239013672 seconds
Concurrent Requests (Time to Complete): 0.5026059150695801 seconds
